In [1]:
import warnings
import logging
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
import mlflow
import mlflow.sklearn

In [2]:
#process log tracking
logging.basicConfig(level=logging.DEBUG,
                    filename='./logfile.log',
                    filemode='w', # 'w' 表示寫模式, 'a' 表示追加模式, 'w' 表示如果文件已存在，先将其清空。如果你想在不清空现有日志的情况下向文件追加日志，可以使用 'a' 模式。
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

logger = logging.getLogger(__name__)


In [3]:
#evaluation function
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2


## RAW data preparation and split data for training

In [4]:
warnings.filterwarnings("ignore")
np.random.seed(40)

# Read the wine-quality csv file from local
logger.debug("raw data ingestion")
data = pd.read_csv("data/red-wine-quality.csv")
data.to_csv("data/red-wine-quality.csv", index=False)

# Split the data into training and test sets. (0.75, 0.25) split.
logger.debug("data prep for training and testing")
train, test = train_test_split(data)

# The predicted column is "quality" which is a scalar from [3, 9]
train_x = train.drop(["quality"], axis=1)
test_x = test.drop(["quality"], axis=1)
train_y = train[["quality"]]
test_y = test[["quality"]]

## Model training and prediction

如果沒有設定實驗追蹤路徑，預設會建立 mlruns 資料夾儲存實驗相關資訊

In [5]:

# hyper parameter tuning
alpha = 0.3
l1_ratio = 0.3

logger.debug("Model training")
exp = mlflow.set_experiment(experiment_name="experment_1")

with mlflow.start_run(experiment_id=exp.experiment_id):
    lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
    lr.fit(train_x, train_y)

    predicted_qualities = lr.predict(test_x)

    (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

    print("Elasticnet model (alpha={:f}, l1_ratio={:f}):".format(alpha, l1_ratio))
    print("  RMSE: %s" % rmse)
    print("  MAE: %s" % mae)
    print("  R2: %s" % r2)

    mlflow.log_param("alpha", alpha)
    mlflow.log_param("l1_ratio", l1_ratio)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)
    mlflow.log_metric("mae", mae)
    mlflow.sklearn.log_model(lr, "mymodel")


Elasticnet model (alpha=0.300000, l1_ratio=0.300000):
  RMSE: 0.7442929001520973
  MAE: 0.5763000946156918
  R2: 0.21508707276848893


## MLflow logging function
1. get tracking uri -> 設定ML實驗儲存位置
2. set tracking uri

In [8]:
# set tracking folder
mlflow.set_tracking_uri(uri="./mytracks")

# 全路徑寫法 file:xxxx
# mlflow.set_tracking_uri(uri=r"file:C:\Users\xdxd2\Sunny_VS_worksapce\Sunny_python\ML\mytracks")

print("The set tracking uri is ", mlflow.get_tracking_uri())

The set tracking uri is  file:C:\Users\xdxd2\Sunny_VS_worksapce\Sunny_python\ML\mytracks


In [9]:

# set experiment nam 
exp = mlflow.set_experiment(experiment_name="experment_for_url")

with mlflow.start_run(experiment_id=exp.experiment_id):
    lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
    lr.fit(train_x, train_y)

    predicted_qualities = lr.predict(test_x)

    (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

    print("Elasticnet model (alpha={:f}, l1_ratio={:f}):".format(alpha, l1_ratio))
    print("  RMSE: %s" % rmse)
    print("  MAE: %s" % mae)
    print("  R2: %s" % r2)

    mlflow.log_param("alpha", alpha)
    mlflow.log_param("l1_ratio", l1_ratio)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)
    mlflow.log_metric("mae", mae)
    mlflow.sklearn.log_model(lr, "mymodel")


2024/01/28 20:48:28 INFO mlflow.tracking.fluent: Experiment with name 'experment_for_url' does not exist. Creating a new experiment.


Elasticnet model (alpha=0.300000, l1_ratio=0.300000):
  RMSE: 0.7442929001520973
  MAE: 0.5763000946156918
  R2: 0.21508707276848893
